In [1]:
def loadLibFolder (folder):
    import os, sys
    if folder not in sys.path:
        sys.path.insert(1, os.path.join(sys.path[0], folder))

# Experimenting with POS dependency parser
To be able to predict a category out of a sentence/text it is assumed that the POS tags and the dependency tree could have an inpact on the result. Here we investigate that relation

In [2]:
from urllib import request, parse
import json
url = 'http://localhost:1337/sentence/'

## Sample text to try out the parser

In [3]:
def parseSentence(sentence):
    try:
        sentence = request.quote(sentence)
        f =  request.urlopen(url + sentence)
        res = json.loads(f.read().decode('latin1'))
        return res
    except:
        return {'sentenceData': []}
def onlyNounsAndVerbs(data):
    return {
        'sentenceData': [word for word in data['sentenceData'] if 'NN' in word['tag'].split('|') or 'VB' in word['tag'].split('|')]
    }
def untilLevel(level, data):
    return {
        'sentenceData': [word for word in data['sentenceData'] if (int)(word['parent']) <= level]
    }
def toWordArray(data):
    return [word['base_word'] for word in data['sentenceData']]

In [4]:
res = parseSentence('Han ler mot henne och hela hans ansikte säger att han älskar henne med hela sitt hjärta')

In [5]:
# Example filtering
print ("Raw data:")
print (res)
print ("All words:")
print ([word['word'] for word in res['sentenceData']])
print ("Level three data:")
print ([word['word']+ '::' + word['tag'].split('|')[0] for word in res['sentenceData'] if (int)(word['parent']) <= 3])
print ("Only nouns and verbs:")
print ([word['word'] for word in res['sentenceData'] if 'NN' in word['tag'].split('|') or 'VB' in word['tag'].split('|')])

print(" ".join(toWordArray(untilLevel(3, onlyNounsAndVerbs(parseSentence(res))))))

Raw data:
{'sentenceData': [{'identifier': '1', 'word': 'han', 'base_word': 'han', 'tag': 'PN|UTR|SIN|DEF|SUB', 'parent': '2'}, {'identifier': '2', 'word': 'ler', 'base_word': 'le', 'tag': 'VB|PRS|AKT', 'parent': '0'}, {'identifier': '3', 'word': 'mot', 'base_word': 'mot', 'tag': 'PP', 'parent': '2'}, {'identifier': '4', 'word': 'henne', 'base_word': 'hon', 'tag': 'PN|UTR|SIN|DEF|OBJ', 'parent': '3'}, {'identifier': '5', 'word': 'och', 'base_word': 'och', 'tag': 'KN', 'parent': '8'}, {'identifier': '6', 'word': 'hela', 'base_word': 'hel', 'tag': 'JJ|POS|UTR/NEU|SIN|DEF|NOM', 'parent': '8'}, {'identifier': '7', 'word': 'hans', 'base_word': 'hans', 'tag': 'PS|UTR/NEU|SIN/PLU|DEF', 'parent': '8'}, {'identifier': '8', 'word': 'ansikte', 'base_word': 'ansikte', 'tag': 'NN|NEU|SIN|IND|NOM', 'parent': '9'}, {'identifier': '9', 'word': 'säger', 'base_word': 'säga', 'tag': 'VB|PRS|AKT', 'parent': '3'}, {'identifier': '10', 'word': 'att', 'base_word': 'att', 'tag': 'SN', 'parent': '9'}, {'identi

## Classification experiment

In [2]:
loadLibFolder('../gensim')

import os
import gensim
import gensim_documents
import dotenv
import numpy as np
dotenv.load()

C:\Users\desktop-godesity\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using Theano backend.
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.


In [3]:
limit_per_category = 2000
use_cache = True
use_all_data = True

In [14]:
categories = []
x_data = []
y_data = []
timesteps = 20
model = gensim.models.Doc2Vec.load(dotenv.get('DOC2VEC_MODEL'))

if use_cache and os.path.isfile('data/tmp_dependency_data'):
    with open('data/tmp_dependency_data_lvl4', 'r', encoding='utf-8', errors='ignore') as tmp_cache_file:
        for category in tmp_cache_file:
            category = category[:-1]
            if category == "\n": continue
            if category not in categories:
                print ("TT", category)
                categories.append(category)
            sentVecs = []
            while True:
                sentence = tmp_cache_file.readline()[:-1]
                if sentence == "":
                    break
                
                # sentVecs.append(gensim.matutils.unitvec(model.infer_vector(doc_words=sentence.split())))
                sentVecs.append([words.index(_words[iw]) + 1 if iw < len(_words) else 0 for iw in range(timestep_range)])
            
            stepsToAdd = max([0, timesteps - len(sentVecs)])
            for i in range(stepsToAdd):
                if stepsToAdd <= 0: break
                sentVecs.append(np.zeros(300))
            y_data.append(categories.index(category))
            x_data.append(sentVecs[:timesteps])
else:
    data = gensim_documents.MMDBDocumentLists(dotenv.get('ARTICLE_PATH', '.') + '/csv_by_category/', useHeading=True, limit=limit_per_category)
    tmp_cache_file = None
    words = []
    if not use_all_data: tmp_cache_file =  open('data/tmp_dependency_data', 'w', encoding='utf-8', errors='ignore')
    for i, doc in enumerate(data):
        if not doc.category in categories:
            categories.append(doc.category)
        if tmp_cache_file != None:
            tmp_cache_file.write(doc.category + "\n")

        sentences = doc.content.split(".")
        sentVecs = []
        timestep_range = 100
        for j in range(timesteps):
            if j >= len(sentences): 
                sentVecs.append(np.zeros(timestep_range))
                continue
            if use_all_data:
                sentence = sentences[j]
            else:
                sentence = " ".join(toWordArray(untilLevel(3, onlyNounsAndVerbs(parseSentence(sentences[j])))))
            if sentence == "":
                sentVecs.append(np.zeros(timestep_range))
                continue
            _words = sentence.split()
            for w in _words:
                if w not in words:
                    words.append(w)
            #artvec = gensim.matutils.unitvec(model.infer_vector(doc_words=_words))
            sentVecs.append([words.index(_words[iw]) + 1 if iw < len(_words) else 0 for iw in range(timestep_range)])
            if tmp_cache_file != None:
                tmp_cache_file.write(sentence + "\n")
        
        if tmp_cache_file != None:
            tmp_cache_file.write("\n")
        x_data.append(sentVecs)
        y_data.append(categories.index(doc.category))

        if i % (limit_per_category/4) == 0 and i != 0:
            print ("New epoch started, nr.", i, "of", len(categories) * limit_per_category, "epochs", float(i) / float(len(categories) * limit_per_category), " %")


New epoch started, nr. 500 of 14000 epochs 0.03571428571428571  %
New epoch started, nr. 1000 of 14000 epochs 0.07142857142857142  %
New epoch started, nr. 1500 of 14000 epochs 0.10714285714285714  %
New epoch started, nr. 2000 of 14000 epochs 0.14285714285714285  %
New epoch started, nr. 2500 of 14000 epochs 0.17857142857142858  %
New epoch started, nr. 3000 of 14000 epochs 0.21428571428571427  %
New epoch started, nr. 3500 of 14000 epochs 0.25  %
New epoch started, nr. 4000 of 14000 epochs 0.2857142857142857  %
New epoch started, nr. 4500 of 14000 epochs 0.32142857142857145  %
New epoch started, nr. 5000 of 14000 epochs 0.35714285714285715  %
New epoch started, nr. 5500 of 14000 epochs 0.39285714285714285  %
New epoch started, nr. 6000 of 14000 epochs 0.42857142857142855  %
New epoch started, nr. 6500 of 14000 epochs 0.4642857142857143  %
New epoch started, nr. 7000 of 14000 epochs 0.5  %
New epoch started, nr. 7500 of 14000 epochs 0.5357142857142857  %
New epoch started, nr. 8000 of

C:\Users\desktop-godesity\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: DeprecationWarning: generator 'MMDBDocumentLists.__iter__' raised StopIteration


In [15]:
min([len(x) for x in x_data])

20

#### Encode one hot vectors for the classes

In [16]:
y_data_one_hot = np.zeros((len(y_data), len(categories)))
y_data_one_hot[np.arange(len(y_data)), np.array(y_data)] = 1


### LSTM classification with keras LSTM cells

In [17]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
import numpy as np

In [18]:
data_dim = len(x_data[0][0])
timesteps = len(x_data[0])
num_classes = len(categories)
n_layers = 10

In [19]:
split = 0.4
limit_train = (int)(len(x_data) * split)
# Generate dummy training data
x_train = x_data[:limit_train]
y_train = y_data_one_hot[:limit_train]

# Generate dummy validation data
x_val = x_data[limit_train:]
y_val = y_data_one_hot[limit_train:]

In [20]:
print(len(x_val))
print(len(x_val[0]))
print(len(x_val[0][0]))

print(len(x_train))
print(len(x_train[0]))
print(len(x_train[0][0]))
print(len(categories))

print(y_train)

8396
20
100
5597
20
100
7
[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 ..., 
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


To train a Sequential LSTM model that can classify a stacked sequence of words we need to define the input as follows:
 * batch_size - number of datapoints in the dataset
 * timesteps - the number of words per sequence
 * data_dim - the number of features per word instance

In [21]:
# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(50, return_sequences=True,
               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 32
for layer in range(n_layers-2):
    model.add(LSTM(50, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(50))  # return a single vector of dimension 32
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

Train on 5597 samples, validate on 8396 samples
Epoch 1/10
5597/5597 [==============================] - 10750s - loss: 1.8573 - acc: 0.2078 - val_loss: 1.9069 - val_acc: 0.2265
Epoch 2/10
5597/5597 [==============================] - 11298s - loss: 1.8049 - acc: 0.2464 - val_loss: 1.8960 - val_acc: 0.2605
Epoch 3/10
5597/5597 [==============================] - 14587s - loss: 1.7841 - acc: 0.2673 - val_loss: 1.9153 - val_acc: 0.2203
Epoch 4/10
5597/5597 [==============================] - 10881s - loss: 1.7633 - acc: 0.2810 - val_loss: 1.8766 - val_acc: 0.2445
Epoch 5/10
5597/5597 [==============================] - 10747s - loss: 1.7497 - acc: 0.2825 - val_loss: 1.9111 - val_acc: 0.2560
Epoch 6/10
5597/5597 [==============================] - 10792s - loss: 1.7369 - acc: 0.2998 - val_loss: 1.9071 - val_acc: 0.2377
Epoch 7/10
5597/5597 [==============================] - 10792s - loss: 1.7305 - acc: 0.3030 - val_loss: 1.8718 - val_acc: 0.2629
Epoch 8/10
5597/5597 [===========================

In [22]:
model.evaluate(x_val, y_val)

8396/8396 [==============================] - 2947s  


[1.8826061483222341, 0.25107193902567942]

In [23]:
prediction = model.predict(np.array(x_val))

In [24]:
copy_prediction = prediction

In [31]:
copy_prediction = [[1.0 if max(y) == i else 0.0 for i in y] for y in prediction]

In [32]:
copy_prediction

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 

In [33]:
y_val

array([[ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       ..., 
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [34]:
from sklearn.metrics import confusion_matrix
import pandas

In [41]:
conf_mat = confusion_matrix([categories[y.argmax()] for y in y_val], [categories[y.argmax()] for y in np.array(copy_prediction)])

In [42]:
pandas.DataFrame(conf_mat, columns=categories, index=categories)

,Allmänt,Blåljus,Ekonomi,Kultur,Nöje,Släkt o vänner,Sport
Allmänt,143,493,111,144,3,139,166
Blåljus,76,312,146,59,7,107,492
Ekonomi,63,162,357,240,11,109,257
Kultur,27,60,247,567,13,123,162
Nöje,29,95,278,448,22,77,251
Släkt o vänner,57,225,188,191,18,194,327
Sport,74,261,84,64,18,186,513


In [50]:
conf_mat = confusion_matrix([categories[y.argmax()] for y in y_val], [categories[0] for y in y_val])
tmp_pd = pandas.DataFrame(conf_mat, columns=['Count'] + categories[1:], index=categories)
pandas.DataFrame(tmp_pd['Count'], columns=['Count'])

,Count
Allmänt,1199
Blåljus,1199
Ekonomi,1199
Kultur,1199
Nöje,1200
Släkt o vänner,1200
Sport,1200
